![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/training/english/classification/ClassifierDL_Train_multi_class_news_category_classifier.ipynb)

# Multi-class Text Classification using ClassifierDL

Only run this block if you are inside Google Colab otherwise skip it

In [ ]:
# Only run this cell when you are using Spark NLP on Google Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)


In [ ]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version; ", spark.version)

Spark NLP version:  4.3.1
Apache Spark version;  3.3.0


Let's download news category dataset for training our text classifier

In [ ]:
!wget -O news_category_train.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_train.csv

--2023-02-20 17:39:41--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_train.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.140.206, 52.216.147.5, 54.231.204.0, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.140.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24032125 (23M) [text/csv]
Saving to: ‘news_category_train.csv’

news_category_train 100%[===================>]  22,92M  10,2MB/s    in 2,3s    

2023-02-20 17:39:44 (10,2 MB/s) - ‘news_category_train.csv’ saved [24032125/24032125]



In [ ]:
!wget -O news_category_test.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_test.csv

--2023-02-20 17:39:46--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_test.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.131.61, 52.217.231.232, 54.231.235.104, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.131.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1504408 (1,4M) [text/csv]
Saving to: ‘news_category_test.csv’

news_category_test. 100%[===================>]   1,43M  2,10MB/s    in 0,7s    

2023-02-20 17:39:47 (2,10 MB/s) - ‘news_category_test.csv’ saved [1504408/1504408]



In [ ]:
!head news_category_train.csv

category,description
Business," Short sellers, Wall Street's dwindling band of ultra cynics, are seeing green again."
Business," Private investment firm Carlyle Group, which has a reputation for making well timed and occasionally controversial plays in the defense industry, has quietly placed its bets on another part of the market."
Business, Soaring crude prices plus worries about the economy and the outlook for earnings are expected to hang over the stock market next week during the depth of the summer doldrums.
Business," Authorities have halted oil export flows from the main pipeline in southern Iraq after intelligence showed a rebel militia could strike infrastructure, an oil official said on Saturday."
Business," Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections."
Business," Stocks ended slightly higher on Friday but stayed near lows for the year as oil prices surged past  #36;

The content is inside `description` column and the labels are inside `category` column

In [ ]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("news_category_train.csv")

In [ ]:
trainDataset.show()

+--------+--------------------+
|category|         description|
+--------+--------------------+
|Business| Short sellers, W...|
|Business| Private investme...|
|Business| Soaring crude pr...|
|Business| Authorities have...|
|Business| Tearaway world o...|
|Business| Stocks ended sli...|
|Business| Assets of the na...|
|Business| Retail sales bou...|
|Business|" After earning a...|
|Business| Short sellers, W...|
|Business| Soaring crude pr...|
|Business| OPEC can do noth...|
|Business| Non OPEC oil exp...|
|Business| WASHINGTON/NEW Y...|
|Business| The dollar tumbl...|
|Business|If you think you ...|
|Business|The purchasing po...|
|Business|There is little c...|
|Business|The US trade defi...|
|Business|Oil giant Shell c...|
+--------+--------------------+
only showing top 20 rows



In [ ]:
trainDataset.count()

120000

In [ ]:
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [ ]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("category")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [ ]:
pipelineModel = pipeline.fit(trainDataset)

In [ ]:
!cd ~/annotator_logs && ls -l

total 148
-rw-r--r-- 1 root root 456 20. Feb 17:41 ClassifierDLApproach_0375e3a8df00.log
-rw-r--r-- 1 root root 918 20. Feb 17:38 ClassifierDLApproach_6fdb8a569309.log
-rw-r--r-- 1 root root 446 20. Feb 15:55 ClassifierDLApproach_97ff5c76d735.log
-rw-r--r-- 1 root root 438 20. Feb 17:38 ClassifierMetrics_09bd6fa2ecf7.log
-rw-r--r-- 1 root root 317 10. Feb 16:54 ClassifierMetrics_17606bbb7d1f.log
-rw-r--r-- 1 root root 313 10. Feb 16:54 ClassifierMetrics_1a6c515483ae.log
-rw-r--r-- 1 root root 441 20. Feb 17:38 ClassifierMetrics_1e0c8ea78e67.log
-rw-r--r-- 1 root root 323 10. Feb 16:54 ClassifierMetrics_2530315112a8.log
-rw-r--r-- 1 root root 445 20. Feb 17:38 ClassifierMetrics_2b7b458fc84d.log
-rw-r--r-- 1 root root 314 10. Feb 16:54 ClassifierMetrics_3ccf43933a23.log
-rw-r--r-- 1 root root 449 20. Feb 17:38 ClassifierMetrics_4a2e4a7dac7c.log
-rw-r--r-- 1 root root 325 10. Feb 16:54 ClassifierMetrics_55c7e364bf2b.log
-rw-r--r-- 1 root root 426 20. Feb 17:38 ClassifierMetrics_73fa92fe4b

In [ ]:
!cat ~/annotator_logs/ClassifierDLApproach_d82e68030034.log

cat: /home/root/annotator_logs/ClassifierDLApproach_d82e68030034.log: No such file or directory


# How to use already trained ClassifierDL pipeline or its model

We have two ways of using what we already trained: pipeline or model.

Let's see how we can save the entire pipeline, load it, and do some prediction with that pre-trained pipeline.

## Save and load pre-trained ClassifierDL pipeline

In [ ]:
# Google Colab is free so it comes with a little memory. 
# It's not possible to save and load in this notebook. But you can do this locally or in a decent machine!

# pipelineModel.save("./classifierdl_pipeline")
# loadedPipeline = PipelineModel.load("./classifierdl_pipeline")
# loadedPipeline.transform(YOUR_DATAFRAME)

# Save and load pre-trained ClassifierDL model

In [ ]:
# dbfs:/ or hdfs:/ if you are saving it on distributed file systems
pipelineModel.stages[-1].write().overwrite().save('./tmp_classifierDL_model')



Let's use our pre-trained ClassifierDLModel in a pipeline: 

In [ ]:

# In a new pipeline you can load it for prediction
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

classsifierdl = ClassifierDLModel.load("./tmp_classifierDL_model") \
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")

pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


Now let's load it back so we can have prediction all together with everything in that pipeline:

In [ ]:
from pyspark.sql.types import StringType

dfTest = spark.createDataFrame([
    "Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.",
    "Scientists have discovered irregular lumps beneath the icy surface of Jupiter's largest moon, Ganymede. These irregular masses may be rock formations, supported by Ganymede's icy shell for billions of years..."
], StringType()).toDF("description")

In [ ]:
prediction = pipeline.fit(dfTest).transform(dfTest)

In [ ]:
prediction.select("class.result").show()

prediction.select("class.metadata").show(truncate=False)

+----------+
|    result|
+----------+
|[Business]|
|[Sci/Tech]|
+----------+

+----------------------------------------------------------------------------------------------------------------+
|metadata                                                                                                        |
+----------------------------------------------------------------------------------------------------------------+
|[{Sports -> 9.7500305E-8, Business -> 0.9999994, World -> 3.060714E-7, Sci/Tech -> 1.7089135E-7, sentence -> 0}]|
|[{Sports -> 5.5346327E-15, Business -> 1.8843019E-13, World -> 8.666048E-14, Sci/Tech -> 1.0, sentence -> 0}]   |
+----------------------------------------------------------------------------------------------------------------+



# Evaluation 

Let's evaluatte our ClassifierDL model we trained earlier, saved it, and loaded it into a new pipeline by using a test dataset that model has never seen:

In [ ]:
testDataset = spark.read \
      .option("header", True) \
      .csv("news_category_test.csv")

In [ ]:
preds = pipelineModel.transform(testDataset)

In [ ]:
preds.select('category','description',"class.result").show(50, truncate=50)

+--------+--------------------------------------------------+----------+
|category|                                       description|    result|
+--------+--------------------------------------------------+----------+
|Business|Unions representing workers at Turner   Newall ...|[Business]|
|Sci/Tech| TORONTO, Canada    A second team of rocketeers...|[Sci/Tech]|
|Sci/Tech| A company founded by a chemistry researcher at...|[Sci/Tech]|
|Sci/Tech| It's barely dawn when Mike Fitzpatrick starts ...|[Sci/Tech]|
|Sci/Tech| Southern California's smog fighting agency wen...|[Sci/Tech]|
|Sci/Tech|"The British Department for Education and Skill...|   [World]|
|Sci/Tech|"confessed author of the Netsky and Sasser viru...|[Sci/Tech]|
|Sci/Tech|\\FOAF/LOAF  and bloom filters have a lot of in...|[Sci/Tech]|
|Sci/Tech|"Wiltshire Police warns about ""phishing"" afte...|[Sci/Tech]|
|Sci/Tech|In its first two years, the UK's dedicated card...|[Sci/Tech]|
|Sci/Tech| A group of technology companies  includi

In [ ]:
preds_df = preds.select('category','description',"class.result").toPandas()

In [ ]:
# The result is an array since in Spark NLP you can have multiple sentences.
# This means you can add SentenceDetector in the pipeline and feed it into
# UniversalSentenceEncoder and you can have prediction based on each sentence.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [ ]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

Let's use `classification_report` from `sklearn` to evaluate the final scores. (keep in mind due to limited resources on a free Google Colab we only used 5 Epochs :)

In [ ]:
print (classification_report(preds_df['result'], preds_df['category']))

              precision    recall  f1-score   support

    Business       0.84      0.85      0.85      1860
    Sci/Tech       0.89      0.84      0.87      2001
      Sports       0.98      0.96      0.97      1944
       World       0.87      0.92      0.90      1795

    accuracy                           0.89      7600
   macro avg       0.89      0.90      0.89      7600
weighted avg       0.90      0.89      0.89      7600

